In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
# plt.style.use('fivethirtyeight')
# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')
sns.set(style='whitegrid',color_codes=True)
%matplotlib inline  


# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
plt.rcParams['axes.unicode_minus'] = False

# matplotlib 한글 폰트 출력코드
from matplotlib import font_manager, rc
import platform
try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass


import warnings
warnings.filterwarnings('ignore')

import time, datetime


In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error

In [127]:
train = pd.read_csv('중급자_train.csv')
test = pd.read_csv('중급자_test.csv')


In [5]:
# searchfor = ['공항', '병원','공단','아파트','시청','터미널','시장']

# a['s_name'][a['s_name'].str.contains('|'.join(searchfor))]

# a= pd.DataFrame(train['station_name'].value_counts())
# a=a.reset_index()
# a.columns=['s_name','geton_off']
# a

In [6]:
# b=a[a['geton_off']>=500]['station_name']

# a= pd.DataFrame(train['station_name'].value_counts())
# a=a.reset_index()
# a.columns=['s_name','geton_off']
# a
# all = ['공항', '병원','공단','아파트','시청','터미널','시장','오거리','동']

# train[train['station_name'].str.contains('|'.join(all))]

# hospital = b[b.str.contains('병원')]

# school = b[b.str.contains('학교')]



### Start

### 1. 합계와 평균 변수 생성 


In [149]:
train['sum']=train['6_8_ride']+train['8_10_ride']+train['10_12_ride']+train['6_8_off']+train['8_10_off']+train['10_12_off']
train['avg']=train['sum']/12

In [150]:
station_sum_top=train.groupby(['station_code','station_name'])['sum','avg'].sum().sort_values(['sum'],ascending=False)

In [151]:
#상위 100개 
pd.set_option('display.max_rows', 100)
station_sum_top.head(100)

,,sum,avg
station_code,station_name,,
3270,제주시청(아라방면),34695.0,2891.250000
3271,제주시청(광양방면),31482.0,2623.500000
357,한라병원,30438.0,2536.500000
2972,제주국제공항(종점),26776.0,2231.333333
358,한라병원,26076.0,2173.000000
1564,제주시외버스터미널,24234.0,2019.500000
149,제주버스터미널,21855.0,1821.250000
1957,중앙로터리,19569.0,1630.750000
638,제주국제공항(구제주방면),18994.0,1582.833333


In [152]:
#인덱스를 칼럼으로 되돌리기
station_sum_top=station_sum_top.reset_index(['station_code','station_name'])

In [153]:
# 저장 및 불러오기
station_sum_top.to_csv('station_sum_top.csv',index=False)
station_sum_top=pd.read_csv('station_sum_top.csv')

### 2. 리스트에 있는 단어들로 데이터 프레임 만들기

In [590]:
#['학교', '공항', '터미널', '오거리', '병원', '아파트', '환승', '로터리','중앙','마트','제주','시장','사거리','약국','공단']
name_list=station_sum_top['station_name'].tolist()
# name_list

In [597]:
name_keyword=['중앙','시장','공단','사거리']
# name_keyword

In [598]:
#리스트에 있는 단어 1 없으면 0
train['have_keyword'] = np.where(train['station_name'].str.contains('|'.join(name_keyword)),1,0)

In [599]:
# keyword 보유로 df 분리함
have_keyword_df = train[train['have_keyword']== 1]
not_keyword_df = train[train['have_keyword']== 0]

In [600]:
#shape 확인
print(have_keyword_df.shape)
print(not_keyword_df.shape)

(24659, 52)
(390764, 52)


### 3. 상관계수 및 회귀계수 그려보기

In [601]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride'])  #['중앙','시장','공단','사거리']

0.7639434595423523

In [602]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride'])

0.6175266604358565

In [450]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) #0.7639434595423523 ['중앙','시장','공단','사거리']

0.7639434595423523

In [451]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) #0.6147442016376625

0.6175266604358565

In [395]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) #0.7589699533389737 ['중앙','시장','공단']

0.7589699533389737

In [396]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride'])#0.6195110234853409

0.6195110234853409

In [354]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) # 075850392110679 ['중앙','시장']
# have_keyword_df.corrwith(have_keyword_df['18~20_ride'])

0.75850392110679

In [353]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) #0.6195632530468766

In [343]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) #0.6901797586536403['공항','터미널','환승','공단','로터리','중앙','시장']

0.6901797586536403

In [344]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) #0.594581969161013

0.594581969161013

In [320]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride'])# 0.689091044225113 ['공항','터미널','환승','공단','로터리','중앙']

0.6890910442251131

In [321]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) #0.5944055162725489

0.5944055162725489

In [303]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) #0.6813977247836832 ['공항','터미널','환승','공단','로터리']

0.6813977247836832

In [304]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride'])  #0.6036562272923014

0.6036562272923014

In [274]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride']) #0.6718813123728677 ['공항','터미널','환승','공단']

0.6718813123728677

In [275]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) 0.6151378301128935

0.6151378301128935

In [263]:
have_keyword_df['sum'].corr(have_keyword_df['18~20_ride'])  #0.6712743978189013 [ '공항','터미널','환승']

0.6712743978189013

In [264]:
not_keyword_df['sum'].corr(not_keyword_df['18~20_ride']) #0.6152019898704613 

0.6152019898704613

In [252]:
airport = have_keyword_df.corrwith(have_keyword_df['18~20_ride'])
airport

id                    0.009679
day                   0.015303
holi                 -0.040267
bus_route_id         -0.123615
in_out                     NaN
station_code         -0.028239
latitude              0.007098
longitude            -0.042192
6~7_ride              0.151528
7~8_ride              0.284767
8~9_ride              0.417574
9~10_ride             0.465184
10~11_ride            0.486450
11~12_ride            0.492322
6~7_takeoff           0.039709
7~8_takeoff           0.153452
8~9_takeoff           0.179805
9~10_takeoff          0.168410
10~11_takeoff         0.200657
11~12_takeoff         0.192122
18~20_ride            1.000000
dis_jejusi           -0.010522
dis_seoquipo          0.001865
dis_jeju             -0.007517
dis_gosan            -0.038713
dis_seongsan          0.050189
dis_po                0.005268
temp                 -0.019527
rain                  0.001829
wind                  0.004344
weekday_0             0.017033
weekday_1             0.021046
weekday_